In [41]:
import numpy as np
import os
from keras.models import Model
from keras import layers
import tensorflow as tf
from keras.backend import manual_variable_initialization
manual_variable_initialization(True)

In [42]:
def _parse_function(proto):

    keys_to_features = {'sound': tf.FixedLenFeature([], tf.string),
                        "label": tf.FixedLenFeature([], tf.float32)}
    
    # Load one example
    parsed_features = tf.parse_single_example(proto, keys_to_features)
    
    # Turn the sound string back into an array
    parsed_features['sound'] = tf.decode_raw(parsed_features['sound'], tf.float32)
    
    
    return parsed_features['sound'], parsed_features["label"]

In [83]:
def create_dataset(filepath, classification=True):
    
    files = [filepath + "/" + file for file in os.listdir(filepath)]

    dataset = tf.data.TFRecordDataset(files)
    
    # Maps the parser on every filepath in the array. You can set the number of parallel loaders here
    dataset = dataset.map(_parse_function, num_parallel_calls=4)
    
    # This dataset will go on forever
    dataset = dataset.repeat()
    
    # Set the number of datapoints you want to load and shuffle 
    #dataset = dataset.shuffle(512)
    
    # Set the batchsize
    dataset = dataset.batch(batchsize)
    
    # Create an iterator
    iterator = dataset.make_one_shot_iterator()
    
    # Create your tf representation of the iterator
    sound, label = iterator.get_next()

    # reshape
    sound = tf.reshape(sound, [-1, 24000, 1])
    label = tf.reshape(label, [-1, 1])
    
#     Create a one hot array for your labels
#     label = tf.one_hot(label, NUM_CLASSES)

    
    return sound, label

In [84]:
def getLabels(x):
    if (x>=0.5):
        label = "CY"
    else:
        label = "CN"
    return(label)
    

In [85]:
test_filepath = "./data/tf_data/test"
validation_filepath = "./data/tf_data/validation"
train_filepath = "./data/tf_data/train"



test_samples = sum([int(file.split("_")[0]) for file in os.listdir(test_filepath)])
validation_samples = sum([int(file.split("_")[0]) for file in os.listdir(validation_filepath)])
train_samples = sum([int(file.split("_")[0]) for file in os.listdir(train_filepath)])


batchsize = train_samples
print("test samples=",test_samples)
print("validation samples =",validation_samples)
print("train samples =",train_samples)

test samples= 8008
validation samples = 5990
train samples = 26007


In [115]:
batchsize = 55
sound_test, label_test = create_dataset(test_filepath)
sound_validation, label_validation = create_dataset(validation_filepath)
sound_train, label_train = create_dataset(train_filepath)


sound_test.shape

TensorShape([Dimension(None), Dimension(24000), Dimension(1)])

In [88]:
#from keras.models import load_model
#model = load_model('./trained_models/model_paper/model.h5')
# inputs = layers.Input(shape=(data.shape[1],1))
inputs = layers.Input(tensor=sound_train)

conv1 = layers.Conv1D(filters=64, kernel_size=129, padding="same")(inputs)
pool1 = layers.MaxPool1D(8)(conv1)

reshape = layers.Reshape((3000,64,1))(pool1)

conv2 = layers.Conv2D(filters=96, kernel_size=(61,1), padding="same", activation='relu')(reshape)
#conv2 = layers.Conv2D(filters=10, kernel_size=(61,64), padding="same", activation='relu')(reshape)

pool2 = layers.MaxPool2D((8,1))(conv2)
conv3 = layers.Conv2D(filters=128, kernel_size=(35,1), padding="same", activation='relu')(pool2)
pool3 = layers.MaxPool2D((4,1))(conv3)
conv4 = layers.Conv2D(filters=160, kernel_size=(17,1), padding="same", activation='relu')(pool3)
pool4 = layers.MaxPool2D((12,1))(conv4)
avgpool = layers.AveragePooling2D((5,1))(pool4)

flatten = layers.Flatten()(avgpool)

dense1 = layers.MaxoutDense(output_dim=128,nb_feature=4)(flatten)
dropout1 = layers.Dropout(0.5)(dense1)
dense2 = layers.MaxoutDense(output_dim=64, nb_feature=4)(dropout1)
dropout2 = layers.Dropout(0.5)(dense2)

# max_out = maximum([Dense(neurons, **dense_args)(input_layer) for _ in range(n_pieces)])

output = layers.Dense(1, activation="sigmoid")(dropout2)

In [89]:
model.compile(optimizer='Adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [90]:
model.load_weights('./trained_models/model_paper/model.h5')

# Model

In [91]:
#model = Model(inputs=inputs, outputs=output)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 24000, 1)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 24000, 64)         8320      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 3000, 64)          0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 3000, 64, 1)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3000, 64, 96)      5952      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 375, 64, 96)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 375, 64, 128)      430208    
__________

In [95]:
#prediction = model.predict(sound_test,steps=test_samples/batchsize,batch_size=batchsize)
with tf.Session() as s:
    x = np.reshape(s.run(sound_train[0:1000]),[1000,24000,1])
    y = np.reshape(s.run(label_train[0:1000]),[1000,1])
print(x.shape)


(1000, 24000, 1)


In [116]:
#res=model.predict(x)
#res=model.evaluate(tf.reshape(sound_test,[8008,1,24000,1]),tf.reshape(label_test,[8008,1,1,1]),
#    steps=8008)
sound_test = tf.reshape(sound_test,[8008,1,24000,1])
label_test = tf.reshape(label_test,[8008,1,1])

total_res = []
with tf.Session() as s:
    for st,lt in zip (sound_test, label_test):
        res=model.evaluate(st,lt,
                    steps=1,batch_size=None)
        total_res.append(res)

TypeError: Tensor objects are only iterable when eager execution is enabled. To iterate over this tensor use tf.map_fn.

In [112]:
sound_test.shape

TensorShape([Dimension(8008), Dimension(1), Dimension(24000), Dimension(1)])

In [97]:
res

[5.318207491248846, 0.48]

In [75]:
labelTr = np.vectorize(getLabels)
pre_labels = labelTr(res)
tru_labels = labelTr(y)
tru_labels[1:5]

array([['CN'],
       ['CY'],
       ['CN'],
       ['CN']], dtype='<U2')

In [27]:
from sklearn.metrics import classification_report
metrics = classification_report(tru_labels,pre_labels)
print(metrics)

              precision    recall  f1-score   support

          CN       0.56      0.58      0.57      4451
          CY       0.45      0.43      0.44      3557

    accuracy                           0.51      8008
   macro avg       0.50      0.50      0.50      8008
weighted avg       0.51      0.51      0.51      8008



In [15]:
#prediction = model.predict(sound_test,steps=test_samples/batchsize,batch_size=batchsize)
x = np.reshape(tf.Session().run(sound_validation[0:validation_samples]),[validation_samples,24000,1])
y = np.reshape(tf.Session().run(label_validation[0:validation_samples]),[validation_samples,1])
print(x.shape)



(5990, 24000, 1)


In [16]:
res=model.predict(x)

In [17]:
labelTr = np.vectorize(getLabels)
pre_labels = labelTr(res)
tru_labels = labelTr(y)

In [18]:
from sklearn.metrics import classification_report
metrics = classification_report(tru_labels,pre_labels)
print(metrics)

              precision    recall  f1-score   support

          CN       0.64      0.59      0.61      3802
          CY       0.37      0.42      0.39      2188

    accuracy                           0.53      5990
   macro avg       0.51      0.51      0.50      5990
weighted avg       0.54      0.53      0.53      5990



In [19]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(tru_labels,pre_labels)

In [20]:
cm

array([[2249, 1553],
       [1270,  918]])